# Counterfactuals guided by prototypes on Boston housing dataset

This notebook goes through an example of [prototypical counterfactuals](../methods/CFProto.ipynb) using [k-d trees](https://en.wikipedia.org/wiki/K-d_tree) to build the prototypes. Please check out [this notebook](./cfproto_mnist.ipynb) for a more in-depth application of the method on MNIST using (auto-)encoders and trust scores.

In this example, we will train a simple neural net to predict whether house prices in the Boston area are above the median value or not. We can then find a counterfactual to see which variables need to be changed to increase or decrease a house price above or below the median value.

In [1]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs 
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from sklearn.datasets import load_boston
from alibi.explainers.cfproto import CounterfactualProto
from sklearn.preprocessing import StandardScaler, MinMaxScaler

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.5.1
Eager execution enabled:  False


## Load and prepare Boston housing dataset

In [2]:
boston = load_boston()
data = boston.data
target = boston.target
feature_names = boston.feature_names

Transform into classification task: target becomes whether house price is above the overall median or not

In [3]:
y = np.zeros((target.shape[0],))
y[np.where(target > np.median(target))[0]] = 1

Remove categorical feature

In [4]:
data = np.delete(data, 3, 1)
feature_names = np.delete(feature_names, 3)

Explanation of remaining features:

- CRIM: per capita crime rate by town
- ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS: proportion of non-retail business acres per town
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per USD10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

Standardize data

In [5]:
random.seed(0)
a = list(range(len(data)))
random.shuffle(a)
length = len(a)

Define train and test set

In [6]:
train_x, train_y = data[a[0:int(0.5*length)]], y[a[0:int(0.5*length)]]
query_x, query_y = data[a[int(0.5*length):int(0.75*length)]], y[a[int(0.5*length):int(0.75*length)]]
test_x, test_y = data[a[int(0.75*length):]], y[a[int(0.75*length):]]

In [7]:
scaler = StandardScaler()
strain_x = scaler.fit_transform(train_x)
squery_x = scaler.transform(query_x)
stest_x = scaler.transform(test_x)

In [8]:
otrain_y = to_categorical(train_y)
oquery_y = to_categorical(query_y)
otest_y = to_categorical(test_y)

## Train model

In [9]:
np.random.seed(42)
tf.random.set_seed(42)

In [10]:
def nn_model():
    x_in = Input(shape=(12,))
    x = Dense(40, activation='relu')(x_in)
    x = Dense(40, activation='relu')(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn

In [11]:
nn = nn_model()
nn.summary()
nn.fit(strain_x, otrain_y, batch_size=64, epochs=500, verbose=0)
nn.save('nn_boston.h5', save_format='h5')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
dense (Dense)                (None, 40)                520       
_________________________________________________________________
dense_1 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 82        
Total params: 2,242
Trainable params: 2,242
Non-trainable params: 0
_________________________________________________________________


In [12]:
nn = load_model('nn_boston.h5')
print(nn.evaluate(squery_x, oquery_y))
print(nn.evaluate(stest_x, otest_y))

[0.23143866563600207, 0.8730159]
[0.31804604417695775, 0.8582677]


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


# Train Autoencoder

In [23]:
def ae_model():
    # encoder
    x_in = Input(shape=(12,))
    x = Dense(30, activation='relu')(x_in)
    x = Dense(15, activation='relu')(x)
    encoded = Dense(5, activation=None)(x)
    encoder = Model(x_in, encoded)

    # decoder
    dec_in = Input(shape=(5,))
    x = Dense(15, activation='relu')(dec_in)
    x = Dense(30, activation='relu')(x)
    decoded = Dense(12, activation=None)(x)
    decoder = Model(dec_in, decoded)

    # autoencoder = encoder + decoder
    x_out = decoder(encoder(x_in))
    autoencoder = Model(x_in, x_out)
    autoencoder.compile(optimizer='adam', loss='mse')

    return autoencoder, encoder, decoder

In [25]:
ae, enc, dec = ae_model()
ae.summary()
ae.fit(strain_x, strain_x, batch_size=128, epochs=100, validation_data=(strain_x, strain_x), verbose=1)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
model_4 (Functional)         (None, 5)                 935       
_________________________________________________________________
model_5 (Functional)         (None, 12)                942       
Total params: 1,877
Trainable params: 1,877
Non-trainable params: 0
_________________________________________________________________
Train on 253 samples, validate on 253 samples
Epoch 1/100
253/253 [==============================] - 0s 1ms/sample - loss: 1.0535 - val_loss: 1.0339
Epoch 2/100
253/253 [==============================] - 0s 46us/sample - loss: 1.0282 - val_loss: 1.0106
Epoch 3/100
253/253 [==============================] - 0s 44us/sample - loss: 1.0057 - val_loss: 0.9902
Epoch 4/100
253/253 [=========

In [27]:
ae.save('boston_ae.h5', save_format='h5')
enc.save('boston_enc.h5', save_format='h5')

## Generate counterfactual guided by the nearest class prototype

Original instance:

In [13]:
X = squery_x[1].reshape((1,) + squery_x[1].shape)
shape = X.shape

Run counterfactual:

In [14]:
# define model
nn = load_model('nn_boston.h5')

# initialize explainer, fit and generate counterfactual
cf = CounterfactualProto(nn, shape, use_kdtree=True, theta=10., max_iterations=1000,
                         feature_range=(strain_x.min(axis=0), strain_x.max(axis=0)), 
                         c_init=1., c_steps=10)

cf.fit(strain_x)
explanation = cf.explain(X)

No encoder specified. Using k-d trees to represent class prototypes.


The prediction flipped from 0 (value below the median) to 1 (above the median):

In [15]:
print(f'Original prediction: {explanation.orig_class}')
print('Counterfactual prediction: {}'.format(explanation.cf['class']))

Original prediction: 1
Counterfactual prediction: 0


Let's take a look at the counterfactual. To make the results more interpretable, we will first undo the pre-processing step and then check where the counterfactual differs from the original instance:

In [16]:
orig = scaler.inverse_transform(X)
counterfactual = scaler.inverse_transform(explanation.cf['X'])
delta = counterfactual - orig
for i, f in enumerate(feature_names):
    if np.abs(delta[0][i]) > 1e-4:
        print('{}: {}'.format(f, delta[0][i]))

RM: -1.8724541664123535
DIS: 2.132163354492188
TAX: 20.029296875
PTRATIO: 0.7855649948120114
LSTAT: 12.238415985107421


So in order to increase the house price, the proportion of owner-occupied units built prior to 1940 should decrease by ~11-12%. This is not surprising since the proportion for the observation is very high at 93.6%. Furthermore, the % of the population with "lower status" should decrease by ~5%.

In [17]:
print('% owner-occupied units built prior to 1940: {}'.format(orig[0][5]))
print('% lower status of the population: {}'.format(orig[0][11]))

% owner-occupied units built prior to 1940: 32.0
% lower status of the population: 2.9700000000000006


In [18]:
print(explanation.cf)

{'X': array([[-0.43147114,  2.6884768 , -1.5028751 , -1.101912  , -0.4000601 ,
        -1.2435888 ,  1.8462507 , -0.576646  , -0.73746014, -1.4876009 ,
         0.3680933 ,  0.51315063]], dtype=float32), 'class': 0, 'proba': array([[0.5437768 , 0.45622322]], dtype=float32), 'grads_graph': array([[-0.06895959,  0.        , -8.506165  ,  5.2730227 , -4.0108366 ,
         9.1113615 , -9.770203  ,  6.7364945 , -2.6101213 , -8.591784  ,
        -0.2569145 , 25.308483  ]], dtype=float32), 'grads_num': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])}


Generate the counterfactual explanation for all query instances.

In [ ]:
query_cf = np.zeros_like(squery_x)
query_cf_y = np.zeros(len(squery_x))
for idx in range(len(squery_x)):
    print(idx)
    X = squery_x[idx].reshape((1,) + squery_x[idx].shape)
    explanation = cf.explain(X)
    query_cf[idx] = explanation.cf['X']
    query_cf_y[idx] = explanation.cf['class']

In [ ]:
tmp = np.concatenate((scaler.inverse_transform(query_cf), query_cf_y[:, np.newaxis]), axis = 1)
np.save("boston_housing_query_cf.npy", tmp)

In [ ]:
query_ccf = np.zeros_like(squery_x)
query_ccf_y = np.zeros(len(squery_x))

for idx in range(len(query_cf)):
    print(idx)
    X = query_cf[idx].reshape((1,) + query_cf[idx].shape)
    explanation = cf.explain(X)
    query_ccf[idx] = explanation.cf['X']
    query_ccf_y[idx] = explanation.cf['class']

In [ ]:
tmp1 = np.concatenate((scaler.inverse_transform(query_ccf), query_pnss_y[:, np.newaxis]), axis = 1)
np.save("boston_housing_query_2cf.npy", tmp1)

In [ ]:
print(query_pnss_y)

In [ ]:
print(query_pns_y)

In [ ]:
print(query_pnss[0].max())

In [ ]:
print(query_pns - query_pnss)